In [2]:
import numpy as np
from functools import reduce
from scipy.linalg import eigh
import matplotlib.pyplot as plt
from scipy.sparse import kron, identity, csr_matrix, lil_matrix
from scipy.sparse.linalg import eigsh, eigs
from qutip import Qobj, ptrace
from qutip import commutator as qt_commutator
from tqdm import tqdm
from itertools import product

In [3]:
def partial_trace(rho, keep, dims):
    """Compute the partial trace of a density matrix."""
    keep_dims = np.prod([dims[i] for i in keep])
    trace_dims = np.prod([dims[i] for i in range(len(dims)) if i not in keep])
    rho = rho.reshape([keep_dims, trace_dims, keep_dims, trace_dims])
    return np.trace(rho, axis1=1, axis2=3).reshape([keep_dims, keep_dims])

In [4]:
# debugging partial trace by comparing numpy vs qutip partial trace - 2 and 3 spins (trace out 1 and 2 spins respectively)

rho_i = 1/np.sqrt(2)
rho_s = np.array([[rho_i**2, rho_i**2], [rho_i**2, rho_i**2]])
rho_2 = np.kron(rho_s, rho_s)
rho_3 = np.kron(rho_s, rho_2)
rho_4 = np.kron(rho_s, rho_3)
print(f"rho_s: {rho_s}")
print(f"rho_2: {rho_2}")
print(f"rho_3: {rho_3}")
print(f"rho_4: {rho_4}")

rho_s: [[0.5 0.5]
 [0.5 0.5]]
rho_2: [[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
rho_3: [[0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]]
rho_4: [[0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625
  0.0625 0.0625 0.0625 0.0625 0.0625 0.0625]
 [0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625
  0.0625 0.0625 0.0625 0.0625 0.0625 0.0625]
 [0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625
  0.0625 0.0625 0.0625 0.0625 0.0625 0.0625]
 [0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625
  0.0625 0.0625 0.0625 0

In [5]:
# Trace out the last spin from the 2 spins matrix - 1 spin left

rho_1r = partial_trace(rho_2, [0], [2, 2])
print(f"rho_1r: {rho_1r}")

rho_2_qobj = Qobj(rho_2, dims=[[2, 2],[2, 2]])
rho_1r_qobj = ptrace(rho_2_qobj, [0])
print(f"rho_1r_qobj: {rho_1r_qobj}")
rho_1r_qobj_dense = rho_1r_qobj.full()
print(f"rho_1r_qobj_dense: {rho_1r_qobj_dense}")

rho_1r: [[0.5 0.5]
 [0.5 0.5]]
rho_1r_qobj: Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.5 0.5]
 [0.5 0.5]]
rho_1r_qobj_dense: [[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]]


In [6]:
# Trace out the last spin from the 3 spins matrix - 2 spins left

rho_2rr = partial_trace(rho_3, [0,1], [2, 2, 2])
print(f"rho_1rr: {rho_2rr}")

rho_3_qobj = Qobj(rho_3, dims=[[2, 2, 2], [2, 2, 2]])
rho_2rr_qobj = ptrace(rho_3_qobj, [0,1])
print(f"rho_1rr_qobj: {rho_2rr_qobj}")
rho_2rr_qobj_dense = rho_2rr_qobj.full()
print(f"rho_1rr_qobj_dense: {rho_2rr_qobj_dense}")

rho_1rr: [[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
rho_1rr_qobj: Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
rho_1rr_qobj_dense: [[0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j]
 [0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j]
 [0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j]
 [0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j]]


In [7]:
# Trace out the last 2 spins from the 3 spins matrix - 1 spin left

rho_1rr = partial_trace(rho_3, [0], [2, 2, 2])
print(f"rho_1rr: {rho_1rr}")

rho_3_qobj = Qobj(rho_3, dims=[[2, 2, 2], [2, 2, 2]])
rho_1rr_qobj = ptrace(rho_3_qobj, [0])
print(f"rho_1rr_qobj: {rho_1rr_qobj}")
rho_1rr_qobj_dense = rho_1rr_qobj.full()
print(f"rho_1rr_qobj_dense: {rho_1rr_qobj_dense}")

rho_1rr: [[0.5 0.5]
 [0.5 0.5]]
rho_1rr_qobj: Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.5 0.5]
 [0.5 0.5]]
rho_1rr_qobj_dense: [[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]]


In [12]:
# Trace out the last spin from the 4 spins matrix - 3 spins left

rho_3rrr = partial_trace(rho_4, [0,1,2], [2, 2, 2, 2])
print(f"rho_3rrr: {rho_3rrr}")

rho_4_qobj = Qobj(rho_4, dims=[[2, 2, 2, 2], [2, 2, 2, 2]])
rho_3rrr_qobj = ptrace(rho_4_qobj, [0,1,2])
print(f"rho_3rrr_qobj: {rho_3rrr_qobj}")
rho_3rrr_qobj_dense = rho_3rrr_qobj.full()
print(f"rho_3rrr_qobj_dense: {rho_3rrr_qobj_dense}")

rho_3rrr: [[0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]]
rho_3rrr_qobj: Quantum object: dims=[[2, 2, 2], [2, 2, 2]], shape=(8, 8), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]]
rho_3rrr_qobj_dense: [[0.125+0.j 0.125+0.j 0.125

In [13]:
# Trace out the last 2 spins from the 4 spins matrix - 2 spins left

rho_2rrr = partial_trace(rho_4, [0,1], [2, 2, 2, 2])
print(f"rho_2rrr: {rho_2rrr}")

rho_4_qobj = Qobj(rho_4, dims=[[2, 2, 2, 2], [2, 2, 2, 2]])
rho_2rrr_qobj = ptrace(rho_4_qobj, [0,1])
print(f"rho_2rrr_qobj: {rho_2rrr_qobj}")
rho_2rrr_qobj_dense = rho_2rrr_qobj.full()
print(f"rho_2rrr_qobj_dense: {rho_2rrr_qobj_dense}")

rho_2rrr: [[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
rho_2rrr_qobj: Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
rho_2rrr_qobj_dense: [[0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j]
 [0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j]
 [0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j]
 [0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j]]


In [11]:
N=3
print([2]*N)

[2, 2, 2]
